In [ ]:
#  
#  Author: Tanish Tyagi
#  

# base libraries
import numpy as np
import pandas as pd
import regex as re
import itertools
import sklearn.metrics as sk
from functools import reduce

# deep learning libraries
import torch
import transformers
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# hyperparameter optimization
import optuna
from optuna.samplers import TPESampler
import optuna.visualization.matplotlib as oviz

# file system manipulation
import os
import shutil
from pathlib import Path
import pickle
import copy

# logging
import logging
import time# base libraries
import numpy as np
import pandas as pd
import regex as re
import itertools
import sklearn.metrics as sk
from functools import reduce

# deep learning libraries
import torch
import transformers
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# hyperparameter optimization
import optuna
from optuna.samplers import TPESampler
import optuna.visualization.matplotlib as oviz

# file system manipulation
import os
import shutil
from pathlib import Path
import pickle
import copy

# logging
import logging
import time

## System Configuration

In [ ]:
# define directories
data_dir = Path("Storage/Bert/")
results_dir = Path("Storage/Bert/Results")

In [ ]:
# set seeds to make computations deterministic
np.random.seed(42)
torch.manual_seed(42)

# check CUDA availability
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", "Yes" if cuda_available else "No")

In [ ]:
# configure logging options
logging.basicConfig(level = logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## Loading in Data

In [ ]:
def prepare_data(trial):
    train_data = pd.read_csv(r"Storage/Bert/train_8_11.csv")
    val_data = pd.read_csv(r"Storage/Bert/valid_8_11.csv")
    test_data = pd.read_csv(r"Storage/Bert/test_8_11.csv")
        
    train_data.columns = val_data.columns = test_data.columns = ["PatientID", "text", "labels"]

    return train_data, val_data, test_data

## Defining ClinicalBERT Model with Hyperparameters

In [ ]:
def define_model(trial, trial_dir):
    # hyperparameter tuning
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-4, log = True)
    adam_epsilon = trial.suggest_float("adam_epilson", 1e-8, 1e-4, log = True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 3)
    early_stopping_patience = trial.suggest_int("early_stopping_patience", 1, 3)

    print("- Learning Rate: {}".format(learning_rate))
    print("- Adam Epsilon: {}".format(adam_epsilon)) 
    print("- Training Epochs: {}".format(num_train_epochs))
    print("- Early Stopping Patience: {}".format(early_stopping_patience))

    # define model name
    model_type = "bert"
    model_name = "emilyalsentzer/Bio_ClinicalBERT"
    max_seq_length = 512 

    model_args = ClassificationArgs(

    ## NLP ARGUMENTS
    sliding_window = False,
    learning_rate = learning_rate, # default 4e-5
    adam_epsilon = adam_epsilon, # default 1e-8
    train_batch_size = 8, # default 8
    eval_batch_size = 4, # default 8
    num_train_epochs = num_train_epochs,  # default 1 (number of epochs model will be trained for)
    do_lower_case = False, # default False
    max_seq_length = max_seq_length, # default 128 (maximum sequence length the model will support)
    
    ## TRAINING LOOP
    logging_steps = 50, # default 50
    manual_seed = 1234, # default None (necessary for reproducible results)
    n_gpu = 2, # default 1 (number of GPUs to use)
    save_steps = 2000, # default 2000 (save a model checkpoint at every specified number of steps)
    output_dir = trial_dir, 
    overwrite_output_dir = True, # default False (if True, then the trained model will be saved to the ouput_dir and will overwrite existing saved models in the same directory)
    
    ## EVALUATE DURING TRAINING
    evaluate_during_training = True, # default False
    evaluate_during_training_steps = 2000, # default  2000  
    evaluate_during_training_verbose = True, # default False
    
    ## EARLY STOPPING
    use_early_stopping = True, # default False
    early_stopping_delta = 0, # default 0 (improvement over best_eval_loss necessary to count as a better checkpoint)
    early_stopping_metric = "eval_loss", # default eval_loss 
    early_stopping_metric_minimize = True, # default True
    early_stopping_patience = early_stopping_patience, # default value 3 (terminate training after these many epochs if there is no improvement in early_stopping_metric then early_stopping_delta)
    
    )
    
    # create the classification model
    model = ClassificationModel(
        model_type, model_name,
        num_labels = 3,
        args = model_args,
        use_cuda = cuda_available
    )
    
    return model

## Optuna Training Loop

In [ ]:
def objective(trial):
    # log time
    start_time = time.localtime()

    # log message
    print("\n-------- TRIAL #{} --------".format(trial.number))

    # create output directory
    trial_dir = "Storage/Bert/Results/trial_{}".format(trial.number)
    if os.path.isdir(trial_dir):
        shutil.rmtree(trial_dir)
        print("\n>>> {}: Removing Directory {}\n".format(time.strftime("%H:%M:%S", time.localtime()), trial_dir))
    os.mkdir(trial_dir)

    # log message
    print("\n>>> {}: Preparing Data\n".format(time.strftime("%H:%M:%S", time.localtime())))

    train_data, val_data, test_data = prepare_data(trial)

    assert len(train_data["labels"].unique() == 3)
    
    # save test dataset to file
    f = open(Path(trial_dir, "data_{}.pkl".format(trial.number)), "wb")
    pickle.dump([train_data, val_data, test_data], f)
    f.close()

    # log message
    print("\n>>> {}: Defining Model\n".format(time.strftime("%H:%M:%S", time.localtime())))

    model = define_model(trial, trial_dir)

    # log message
    print("\n>>> {}: Started Training\n".format(time.strftime("%H:%M:%S", time.localtime())))

    # train model
    model.train_model(
        train_data,
        eval_df = val_data,
        # auc = sk.roc_auc_score,
        # acc = sk.accuracy_score
    )

    print("\n>>> {}: Started Evaluation on Validation Set\n".format(time.strftime("%H:%M:%S", time.localtime())))
    
    results, model_outputs, wrong_predictions = model.eval_model(
        val_data,
        # auc = sk.roc_auc_score,
        # acc = sk.accuracy_score
    )

    # save to file
    f = open(Path(trial_dir, "training_results_{}.pkl".format(trial.number)), "wb")
    pickle.dump([model, results, model_outputs, wrong_predictions], f)
    f.close()

    # output message, initialize empty list
    print(">>> {}: Get Sequence Probabilities\n".format(time.strftime("%H:%M:%S", time.localtime())))
    df_list = []

    # extract context window probabilities
    max_prob_list = []
    val_prob_list = []
    val_pred_list = []
    for i in range(len(val_data)):
        # prob_list = list(torch.softmax(torch.from_numpy(model_outputs[i]), axis=0)[:,1])
        prob_list = torch.softmax(torch.from_numpy(model_outputs[i]), axis=0)
        #print("Prob List: ", prob_list, type(prob_list))

        extracted_prob_list = []
        for i in range(len(prob_list)):
            extracted_prob_list.append(float(prob_list[i]))

        #print("Extracted Prob List: ", extracted_prob_list)
        # find max one in each submatrix of length 3
        max_proba = max(extracted_prob_list)

        # identify model prediction based on location of max_proba within extracted_prob_list
        if (extracted_prob_list[0] == max_proba):
            val_pred_list.append(0)
        elif (extracted_prob_list[1] == max_proba):
            val_pred_list.append(1)
        else:
            val_pred_list.append(2)

        max_prob_list.append(max_proba)
        val_prob_list.append(extracted_prob_list)
    
    cw_probs = pd.DataFrame(columns = ["PatientID", "Prob", "Pred"])
    cw_probs["PatientID"] = val_data["PatientID"]
    cw_probs["Prob"] = max_prob_list
    cw_probs["Pred"] = val_pred_list
    cw_probs.to_csv(trial_dir + "/sequence_probabilities{}.csv".format(trial.number))

    # compute metrics
    # print("Shapes of Y-True and Y-Pred", val_data["labels"].shape, cw_probs["Prob"].shape) 
    best_auc = sk.roc_auc_score(val_data["labels"].to_list(), val_prob_list, multi_class = "ovr", average = "weighted")
    # best_auc = get_auc(val_prob_list, val_data["labels"])
    # best_acc, best_threshold = get_best_acc(cw_probs, val_data)
    best_acc = sk.accuracy_score(val_data["labels"].to_list(), cw_probs["Pred"].to_list())
    print(">>> {}: Current AUC: {}\n".format(time.strftime("%H:%M:%S", time.localtime()), best_auc))
    print(">>> {}: Current ACC: {}\n".format(time.strftime("%H:%M:%S", time.localtime()), best_acc))
    #print(">>> {}: Threshold for Validation Accuracy: {}\n".format(time.strftime("%H:%M:%S", time.localtime()), best_threshold))
    print(">>> {}: Start Training Time\n".format(time.strftime("%H:%M:%S", start_time)))
    print(">>> {}: Finish Training Time\n".format(time.strftime("%H:%M:%S", time.localtime())))

    return best_acc

## Optuna Study Creation

In [ ]:
# add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

# unique identifier of the study
study_name = "slat-study" 

# create study database
storage_name = "sqlite:///{}.db".format("Storage/Bert/Results/" + study_name)
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed = 1234, multivariate = True), study_name = study_name, storage = storage_name, load_if_exists = True)
study.optimize(objective, n_trials = 20, gc_after_trial = True)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("\nStudy Statistics:")
print("- Finished Trials: ", len(study.trials))
print("- Pruned Trials: ", len(pruned_trials))
print("- Complete Trials: ", len(complete_trials))

print("\nBest Trial:")
best_trial = study.best_trial

print("- Number: ", best_trial.number)
print("- Value: ", best_trial.value)
print("- Hyperparameters: ")

for key, value in best_trial.params.items():
    print("   - {}: {}".format(key, value))

## Evaluation on Held-Out Test Set

In [ ]:
final_test = pd.read_csv(r"Storage/Bert/test_8_11.csv")
final_test.columns = ["PatientID", "text", "labels"]

### Loading in Model

In [ ]:
best_model = ClassificationModel(
    "bert",
    "Storage/Bert/best_model/trial_0"
)

In [ ]:
test_results, test_outputs, test_wrong = best_model.eval_model(
    final_test,
)

### Applying Softmax to Convert Model output to Probabilities

In [ ]:
max_prob_list = []
test_prob_list = []
test_pred_list = []
for i in range(len(final_test)):
    # prob_list = list(torch.softmax(torch.from_numpy(model_outputs[i]), axis=0)[:,1])
    prob_list = torch.softmax(torch.from_numpy(test_outputs[i]), axis=0)
    #print("Prob List: ", prob_list, type(prob_list))

    extracted_prob_list = []
    for i in range(len(prob_list)):
        extracted_prob_list.append(float(prob_list[i]))

    #print("Extracted Prob List: ", extracted_prob_list)
    # find max one in each submatrix of length 3
    max_proba = max(extracted_prob_list)

    # identify model prediction based on location of max_proba within extracted_prob_list
    if (extracted_prob_list[0] == max_proba):
        test_pred_list.append(0)
    elif (extracted_prob_list[1] == max_proba):
        test_pred_list.append(1)
    else:
        test_pred_list.append(2)

    max_prob_list.append(max_proba)
    test_prob_list.append(extracted_prob_list)

In [ ]:
final_test["pred"] = test_pred_list
final_test["proba"] = max_prob_list

### Accuracy and AUC

In [ ]:
sk.accuracy_score(test_pred_list, final_test["labels"])

In [ ]:
sk.roc_auc_score(final_test["labels"], test_prob_list, multi_class = "ovr", average = "weighted")

### Saving Results

In [ ]:
import pickle

f = open("Storage/Bert/test_set_prob.pkl", "wb")
pickle.dump(str(test_prob_list), f)
f.close()

### Advanced Metrics

In [ ]:
from sklearn import metrics

target_names = ['Negative', 'Neither', 'Positive']
results = metrics.classification_report(final_test["labels"], final_test["pred"], target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = metrics.confusion_matrix(final_test["labels"], final_test["pred"])
print("Micro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "micro"))
print("Macro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "macro"))
print("Weighted F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "weighted"))
print("Confusion Matrix: \n", conf_mat)
print("Classification Report: \n", results)

In [ ]:
FP = conf_mat.sum(axis = 0) - np.diag(conf_mat) 
FN = conf_mat.sum(axis = 1) - np.diag(conf_mat)
TP = np.diag(conf_mat)
TN = conf_mat.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)
print("FPR: ", FPR)

## ROC Curves

In [ ]:
n_class = 3
y_test = final_test["labels"]
test_prob_list = np.array(test_prob_list)

fpr = {}
tpr = {}
thresh = {}
roc_auc = {}

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = metrics.roc_curve(y_test, test_prob_list[:,i], pos_label = i)
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    
lw = 2

# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--', color='green', label='Positive Class: TF-IDF (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
# plt.show()

plt.plot(fpr[1], tpr[1], linestyle='--',color='red', label='Negative Class: TF-IDF (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
# plt.show()

plt.plot(fpr[2], tpr[2], linestyle='--',color='grey', label='Neither Class: TF-IDF (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
# plt.show()
plt.savefig("bert_roc.svg")